# Loading Chains from LangchainHub

In [22]:
import os
import yaml
# Load the config file
with open('../../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Get the value of the environment variable from the config
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

## Loading math chain

In [23]:
from langchain.chains import load_chain
chain = load_chain("lc://chains/llm-math/chain.json")
chain.run("whats 2 raised to .12")



> Entering new LLMMathChain chain...
whats 2 raised to .12
Answer: 1.0791812460476249
> Finished chain.


'Answer: 1.0791812460476249'

## Chain to answer over vector database

In [25]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
import os

In [26]:

from langchain.document_loaders import TextLoader
# https://github.com/hwchase17/chat-your-data/blob/master/state_of_the_union.txt
loader = TextLoader('../../state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(texts, embeddings)

Using embedded DuckDB without persistence: data will be transient


In [27]:
chain = load_chain("lc://chains/vector-db-qa/stuff/chain.json", vectorstore=vectorstore)

/Users/dmaguire/Library/Caches/pypoetry/virtualenvs/llm-tool-sandbox-O45YYzTu-py3.10/lib/python3.10/site-packages/langchain/chains/retrieval_qa/base.py:183: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [28]:
query = "What did the president say about Ketanji Brown Jackson"
chain.run(query)

" President Biden said that Ketanji Brown Jackson is one of the nation's top legal minds, a former top litigator in private practice and a former federal public defender, and comes from a family of public school educators and police officers. He also said she is a consensus builder and has received a broad range of support."

In [31]:
query = "What were the president's key points on foreign policy?"
chain.run(query)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


" The president's key points on foreign policy were that the United States and its allies will defend NATO countries with the full force of their collective power, and that the United States will stand with the Ukrainian people in their struggle against Russia's aggression."